In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re

In [ ]:
job_list_saramin = []
page_num = 1
main_url = 'http://www.saramin.co.kr'

while True:
    url = f'https://www.saramin.co.kr/zf_user/jobs/list/job-category?page={page_num}&cat_mcls=2&search_optional_item=n&search_done=y&panel_count=y&isAjaxRequest=0&page_count=50&sort=RL&type=job-category&is_param=1&isSearchResultEmpty=1&isSectionHome=0&searchParamCount=1#searchTitle'
    req_header = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    }
    
    html = requests.get(url, req_header).text
    soup = BeautifulSoup(html, 'html.parser')
    
    if soup.select('div.list_body div.list_item') == []:
        break
    
    for tag in soup.select('div.list_body div.list_item'):
        company_dict = {}
        company_dict["기업명"] = tag.select('div.company_nm a span')[0].text
        company_dict["직무"] = tag.select('div.notification_info a span')[0].text
        company_dict["경력사항"] = tag.select('div.recruit_condition p.career')[0].text
        company_dict["학력"] = tag.select('div.recruit_condition p.education')[0].text
        
        try:    
            company_dict["근무지"] = tag.select('div.company_info p.work_place')[0].text
        except:
            company_dict["근무지"] = ""
            
        try:
            company_dict["근무형태"] = tag.select('div.company_info p.employment_type')[0].text
        except:
            company_dict["근무형태"] = ""

        # 분류
        category_list = []
        for item in tag.select('div.notification_info div.job_meta span span'):
            category_list.append(item.text)
        company_dict["분류"] = str(category_list)
        
        # 상세링크
        detail_url = tag.select_one('div.notification_info a').attrs['href']
        full_detail_url = urljoin(main_url, detail_url)
        company_dict["상세링크"] = full_detail_url
        
        job_list_saramin.append(company_dict)
    page_num += 1


In [ ]:
# csv로 저장
import pandas as pd
saramin = pd.DataFrame(job_list_saramin)
saramin.to_csv('saramin_recruit.csv', encoding='utf8')